In [1]:
import pandas as pd
import os
import shutil
from datetime import datetime

In [2]:
# User-defined options
rootdir = '/Users/gorkalasso/Documents/GitHub/filo_GP-bat_NPC1/scr/seq_to_feat/'
outdir_root =  rootdir + 'output/5_feature_conversion/'
# feature options:
# ASA_A	ASA_T	AROM	BULK	CHARGE	COIL	FLEX	HYDRO_C	HYDRO_KD	INTER_H	MOLWEI	POL_G
all_feat_ls = ['ASA_T','AROM','BULK','CHARGE','COIL','FLEX',\
               'HYDRO_KD','INTER_H','MOLWEI','POL_G']
dist_ls = [7, 15]

### Functions 

In [3]:
def Feat_2_str(ls):
    s = ''
    for feat in ls:
        if s == '':
            s = feat
        else:
            s += '_' + feat
    return(s)

def Print_initlog(fh_log0, now0, df0, feat_s0, feat_ls0):
    error = 0
    print(now0, file=fh_log0)
    print('Available features', file=fh_log0)
    for count,feat in enumerate(df0.columns):
        print('\t', count, feat, file=fh_log0)
    print('Requested features', file=fh_log0)
    for count, feat in enumerate(feat_ls):
        print('\t', count, feat, file=fh_log)  
    for feat in feat_ls:
        if feat not in df.columns:
            print('Fatal error!, feature', feat, 'does not exists in scale df', sep=' ')
            print('Fatal error!, feature', feat, 'does not exists in scale df', sep=' ', file=fh_log)
            error = 1
            break
    return(error)

def Create_outdir(fh_log0, dir0, dir1):
    print('Outdir', dir1, sep='\t', file = fh_log0)
    if os.path.isdir(dir0):
        print('Outdir 0 exists')
        if os.path.isdir(dir1):
            print('Outdir 1 exists, deleting')
            print('Outdir 1 exists, deleting', file= fh_log0)
            shutil.rmtree(dir1)
        print('Creating outdir 1')
        os.mkdir(dir1)
    else:
        print('Creating outdir 0 & 1')
        os.mkdir(dir0)
        os.mkdir(dir1)    

def Extrac_interres_info(fh_log0, fl):
    fh = open(fl, 'r')
    s = fh.readline().rstrip()
    ls = s.split(', ')
    fh.close()
    print('Feature description size', len(ls), sep='\t')
    print('Feature description size', len(ls), sep='\t', file= fh_log0)
    return(ls)
        
def Convert_inputFasta(fh_log0, infile0, outfile0, df0, feat_ls0, res_info_ls0):
    fh_in = open(infile0, 'r')
    fh_out = open(outfile0, 'w')
    s = 'Label'
    # printing feature description
    for item in res_info_ls0:
            for feat in feat_ls0:
                s += ',' + feat + '_' + item
    print(s, file=fh_out)
    
    head_ct = 0
    for s in fh_in:
        s = s.rstrip()
#         print(s)
        if s[0] == '>':
            head = s[1:]
            head_ct += 1
            print('Entry', head_ct, head, sep='\t', file=fh_log0)
        else:
            seq = s
            print('Sequence', seq, sep='\t', file=fh_log0)
            feat_desc, feat_val = Seq_2_feat(seq, df0, feat_ls0)
            if head_ct == 1:
                feat_len = len(feat_val.split(','))
#             print('Feat values', feat_val, sep='\t', file=fh_log0)
            print('', file=fh_log0)
#             print(head, 'descrp', feat_desc, sep=', ', file=fh_out)
            print(head, feat_val, sep=',', file=fh_out)
    print('Instance counter', head_ct, sep='\t', file=fh_log0)
    print('Feature vector size', feat_len, sep='\t', file=fh_log0)
    print('Instance counter', head_ct, sep='\t')
    print('Feature vector size', feat_len, sep='\t')
    fh_in.close()
    fh_out.close()    
    
def Seq_2_feat(seq0, df0, feat_ls0):
    feat_desc = ''
    feat_val = ''
    feat_ct = 0
    print('Keyw','feat_ct', 'rescount', 'res', 'feat', 'val', sep='\t', file=fh_log)
#     print(df0)
#     print(seq0)
    for rescount, res in enumerate(seq0):
#         print('residue is: ', res)
        for feat in feat_ls0:
            feat_ct += 1
            desc = feat + '_' + res + str(rescount + 1)
            if res == '-':
                val = ''
            else:
                val = df0.at[res,feat]
            if isinstance(val, float):
                val1 = round(val, 1)
                val1 = str(val1)
                val1 = val1.rstrip()
            elif isinstance(val, int):
                val1 = str(val)
                val1 = val1.rstrip()
            else:
                if val != '':
                    val1 = val
                else:
                    val1 = val
                    val1 = val1.rstrip()
            if feat_val == '':
                feat_desc = desc
                feat_val = val1
            else:
                feat_desc += ',' + desc
#                 print(val1)
                feat_val += ',' + val1
            print('Feat', feat_ct, rescount+1, res, feat, val1, sep='\t', file=fh_log)
    return feat_desc, feat_val

## Main

In [4]:
# Initialization
# read master scale table and report what features are available
df = pd.read_excel(rootdir + 'input/0_amino_acid_scales.xlsx')
df.drop(columns=['Unnamed: 0','Resid 3letter'], inplace=True)
df.set_index('Resid 1letter', inplace=True)
display(df)
# date
now = datetime.now() # current date and time
date = now.strftime("%m_%d_%Y")
# print(now)
# print(date)

,ASA_A,ASA_T,AROM,BULK,CHARGE,COIL,FLEX,HYDRO_C,HYDRO_KD,INTER_H,MOLWEI,POL_G
Resid 1letter,,,,,,,,,,,,
G,91.77,104.0,NO,3.40,POL,1.251,0.54,0.36,-0.4,-0.06,75.0,9.0
A,121.32,129.0,NO,11.50,HYDRO,0.824,0.36,0.38,1.8,0.00,89.0,8.1
S,136.12,155.0,NO,9.47,POL,1.130,0.51,0.22,-0.8,-0.08,105.0,9.2
C,149.79,167.0,NO,13.46,POL,0.953,0.35,0.50,2.5,0.33,121.0,5.5
T,158.03,172.0,NO,15.77,POL,1.148,0.44,0.23,-0.7,-0.10,119.0,8.6
D,167.54,193.0,NO,11.68,NEG,1.197,0.51,0.15,-3.5,-0.15,133.0,13.0
V,168.41,174.0,NO,21.57,HYDRO,0.772,0.39,0.54,4.2,0.13,117.0,5.9
P,170.85,159.0,NO,17.43,HYDRO,1.540,0.51,0.18,-1.6,-0.20,115.0,8.0
N,175.17,195.0,NO,12.82,POL,1.167,0.46,0.12,-3.5,-0.13,132.0,11.6


# One fasta per feature in all_feat_ls

In [5]:
# Sort features alphabetically
all_feat_ls = sorted(all_feat_ls, key=str.lower)
for feat in all_feat_ls:
    print(f'FEATURE: {feat}')
    feat_ls = [feat]
    feat_s = Feat_2_str(feat_ls)
    for dist in dist_ls:
        fh_log = open(outdir_root + 'logs/featextract_' + feat_s + '_' + str(dist) + '_date_' + date + '.log', 'w')
        # Initialize
        error = Print_initlog(fh_log, now, df, feat_s, feat_ls)
        if error != 0:
            print('Fatal error 1')
            break
        print('Extracting features for distance', dist)
        # Create directory
        if not os.path.isdir(outdir_root):
            os.mkdir(outdir_root)
        outdir0 = outdir_root + str(dist) + '/'
        outdir1 = outdir0 + feat_s + '/'
        Create_outdir(fh_log, outdir0, outdir1)
        # residue description
        inresinfofile = rootdir + '/output/4_GP_NPC1_interfacial/' + str(dist) + '/interfpos_combined_' + str(dist) + '.txt'
        res_info_ls = Extrac_interres_info(fh_log, inresinfofile)
        # Seq to Features
        inseqfile = rootdir + '/output/4_GP_NPC1_interfacial/' + str(dist) + '/interface_combined_' + str(dist) + '.txt'
        outfile = outdir1 + str(dist) + '_' + feat_s + '.txt'
        Convert_inputFasta(fh_log, inseqfile, outfile, df, feat_ls, res_info_ls)
        print('------------------------------------------------')
        fh_log.close()

FEATURE: AROM
Extracting features for distance 7
Creating outdir 0 & 1
Feature description size	60
Instance counter	664
Feature vector size	60
------------------------------------------------
Extracting features for distance 15
Creating outdir 0 & 1
Feature description size	132
Instance counter	664
Feature vector size	132
------------------------------------------------
FEATURE: ASA_T
Extracting features for distance 7
Outdir 0 exists
Creating outdir 1
Feature description size	60
Instance counter	664
Feature vector size	60
------------------------------------------------
Extracting features for distance 15
Outdir 0 exists
Creating outdir 1
Feature description size	132
Instance counter	664
Feature vector size	132
------------------------------------------------
FEATURE: BULK
Extracting features for distance 7
Outdir 0 exists
Creating outdir 1
Feature description size	60
Instance counter	664
Feature vector size	60
------------------------------------------------
Extracting features for d

In [6]:
print('Finito')

Finito
